In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [43]:
def metric(preds, actuals):
    preds = preds.reshape(-1)
    actuals = actuals.reshape(-1)
    assert preds.shape == actuals.shape
    return 100 * np.linalg.norm((actuals - preds) / actuals) / np.sqrt(preds.shape[0])

def test_da_shit(actuals, preds):
    new_test= pd.DataFrame({'Actuals': actuals,'Preds': preds})
    new_test = new_test.loc[new_test['Actuals'] != 0,:]
    return metric(np.array(new_test['Actuals']), np.array(new_test['Preds']))

In [44]:
train = pd.read_csv('data/train.csv', parse_dates = True)
stores = pd.read_csv('data/store.csv')

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


## Base cleaning

In [48]:
no_null_sales = train.loc[(train['Sales'].notnull()) | (train['Open'] == 0), :]
no_null_sales.loc[:,'Sales'].fillna(0, inplace=True)

no_null_sales = no_null_sales.loc[no_null_sales['Store'].notnull(), :]

no_null_sales.loc[:,'Date'] = pd.to_datetime(no_null_sales.loc[:,'Date'], format='%Y-%m-%d')
no_null_sales.loc[:,'DayOfWeek'] = no_null_sales.loc[:,'Date'].dt.weekday + 1

/opt/anaconda3/envs/minicomp/lib/python3.8/site-packages/pandas/core/series.py:4517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


## Baseline 1

In [55]:
X = no_null_sales[['Store']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = 'Store').loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

40.48447088056098

## Baseline 2

In [56]:
X = no_null_sales[['Store', 'DayOfWeek']]
y = no_null_sales['Sales']
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)
lazy_est = pd.concat([X_train, y_train], axis=1)
lazy_est = lazy_est.groupby(['Store', 'DayOfWeek']).mean()
preds = X_test.merge(lazy_est, how = 'left', on = ['Store', 'DayOfWeek']).loc[:, 'Sales']
actuals = np.array(y_test)
test_da_shit(actuals, preds)

25.392379206905172

## Further cleaning

In [59]:
no_null_sales.loc[no_null_sales['StateHoliday'] == 0.0, 'StateHoliday'] = '0'
no_null_sales.loc[no_null_sales['StateHoliday'] == 0, 'StateHoliday'] = '0'